In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import seaborn as sns

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params); np.random.seed(0)
%matplotlib inline
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load & Pre-process dataset

In [3]:
bg_access = pd.read_csv('../data/bg_transit_adi.csv', dtype={'bg_fips':str})

bg_access = bg_access[['bg_fips', 'address', 'distance_mi', 'ADI_NATRANK', 'ADI_STATERNK', 
                        'bg_state', 'address_state' ,'transit_time', 'walking_time']]

bg_access['bg_county'] = bg_access.progress_apply(lambda x:x.bg_fips[:5], axis=1)

100%|██████████| 232735/232735 [00:03<00:00, 68540.86it/s]


In [4]:
bg_access

,bg_fips,address,distance_mi,ADI_NATRANK,ADI_STATERNK,bg_state,address_state,transit_time,walking_time,bg_county
0,010010201001,"203 N Court St, Prattville, AL 36067",0.835864,73,5,AL,AL,20.080000,20.080000,01001
1,010010201002,"203 N Court St, Prattville, AL 36067",1.684913,62,3,AL,AL,37.300000,37.300000,01001
2,010010202001,"203 N Court St, Prattville, AL 36067",1.065305,83,7,AL,AL,20.330000,20.330000,01001
3,010010202002,"203 N Court St, Prattville, AL 36067",0.327409,87,7,AL,AL,8.400000,8.400000,01001
4,010010203001,"203 N Court St, Prattville, AL 36067",1.367486,73,5,AL,AL,42.630000,42.630000,01001
...,...,...,...,...,...,...,...,...,...,...
232730,550791872003,"7219 S. 27th Street Franklin, WI 53132",2.915941,21,1,WI,WI,131.750000,76.800000,55079
232731,551010002002,"214 7th St, Racine, WI 53403",0.497511,87,10,WI,WI,13.166667,13.166667,55101
232732,551010006004,"2219 Washington Ave, Racine, WI 53405",0.258076,89,10,WI,WI,6.783333,6.783333,55101
232733,551010017025,"2000 De Koven Ave Racine, WI 53403",3.982413,57,5,WI,WI,59.700000,112.566667,55101


2. Find each BG's rural status (based on RUCC 2013 Codes)
- `county_rurality`: the entire county-level dataset of RUCC 2013 codes from USDA ERS, where BGs in counties tagged with all subcategories within 'Metro' are considered as urban and all within 'Nonmetro' as rural. 

In [5]:
county_rurality = pd.read_csv('../data/ruralurbancodes2013.csv', 
                          dtype = {'FIPS':str})

county_rurality['RUCC_2013'] = county_rurality['RUCC_2013'].astype(str)
county_rurality.Description = county_rurality.Description.str.strip()

def decide_urban_degree(text):
    
    if text in ['Metro - Counties in metro areas of 250,000 to 1 million population',
       'Metro - Counties in metro areas of fewer than 250,000 population', 
               'Metro - Counties in metro areas of 1 million population or more']:
        
        return 'Metro'
    
    elif text in ['Nonmetro - Urban population of 20,000 or more, adjacent to a metro area', 
                  'Nonmetro - Urban population of 20,000 or more, not adjacent to a metro area']:
        return 'Nonmetro'
    
    elif text in ['Nonmetro - Urban population of 2,500 to 19,999, not adjacent to a metro area',
                  'Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area']:
        return 'Nonmetro'
    
    elif text in ['Nonmetro - Completely rural or less than 2,500 urban population, not adjacent to a metro area',
                  'Nonmetro - Completely rural or less than 2,500 urban population, adjacent to a metro area']:
        return 'Nonmetro'
    else:
        return 'N/A'


county_rurality['status'] = county_rurality.apply(lambda x:decide_urban_degree(x.Description), axis=1)
county_rurality = county_rurality.rename(columns={'FIPS':'bg_county'})
county_rurality 

,bg_county,State,County_Name,Population_2010,RUCC_2013,Description,status
0,01001,AL,Autauga County,"54,571",2.0,"Metro - Counties in metro areas of 250,000 to ...",Metro
1,01003,AL,Baldwin County,"182,265",3.0,Metro - Counties in metro areas of fewer than ...,Metro
2,01005,AL,Barbour County,"27,457",6.0,"Nonmetro - Urban population of 2,500 to 19,999...",Nonmetro
3,01007,AL,Bibb County,"22,915",1.0,Metro - Counties in metro areas of 1 million p...,Metro
4,01009,AL,Blount County,"57,322",1.0,Metro - Counties in metro areas of 1 million p...,Metro
...,...,...,...,...,...,...,...
3229,72151,PR,Yabucoa Municipio,"37,941",1.0,Metro - Counties in metro areas of 1 million p...,Metro
3230,72153,PR,Yauco Municipio,"42,043",2.0,"Metro - Counties in metro areas of 250,000 to ...",Metro
3231,78010,VI,St. Croix Island,"50,601",5.0,"Nonmetro - Urban population of 20,000 or more,...",Nonmetro
3232,78020,VI,St. John Island,"4,170",7.0,"Nonmetro - Urban population of 2,500 to 19,999...",Nonmetro


3. Merge `bg_access` with the `county_rurality` information, resulting in a dataset of all block groups' FIPS code, their ADI, state/county, and urban/rural status. 

In [6]:
bg_access = bg_access.merge(county_rurality[['bg_county', 'status']].drop_duplicates(subset=['bg_county']), how='left', on='bg_county')

In [7]:
bg_access.describe()

,distance_mi,ADI_NATRANK,ADI_STATERNK,transit_time,walking_time
count,232735.000000,232735.000000,232735.000000,162513.000000,232256.000000
mean,2.563074,50.009711,5.485720,28.237746,84.822415
std,3.165674,28.729023,2.869607,28.539258,364.246471
min,0.000197,1.000000,1.000000,0.000000,0.000000
25%,0.684232,25.000000,3.000000,13.300000,20.030000
50%,1.438754,50.000000,5.000000,23.670000,42.400000
75%,3.077534,75.000000,8.000000,38.530000,90.520000
max,24.990148,100.000000,10.000000,2631.880000,28848.120000


In [9]:
def shorter_travel(transit, walk):
    travel = 0
    if np.isnan(transit) == False:
        if np.isnan(walk) == False:
            if transit >= walk: # transit 0 walk 0
                travel = walk
            else:
                travel = transit
        else: # transit 0 walk x
            travel = transit
    else: #transit x
        if np.isnan(walk) == False: # transit x walk 0
            travel = walk
        else:
            travel = None # transit x walk x

    return travel

In [10]:
bg_access['access'] = bg_access.progress_apply(lambda x:shorter_travel(x.transit_time, x.walking_time), axis=1)

100%|██████████| 232735/232735 [00:07<00:00, 31918.49it/s]


In [11]:
bg_access.describe()

,distance_mi,ADI_NATRANK,ADI_STATERNK,transit_time,walking_time,access
count,232735.000000,232735.000000,232735.000000,162513.000000,232256.000000,232337.000000
mean,2.563074,50.009711,5.485720,28.237746,84.822415,78.562199
std,3.165674,28.729023,2.869607,28.539258,364.246471,338.006703
min,0.000197,1.000000,1.000000,0.000000,0.000000,0.000000
25%,0.684232,25.000000,3.000000,13.300000,20.030000,17.330000
50%,1.438754,50.000000,5.000000,23.670000,42.400000,35.320000
75%,3.077534,75.000000,8.000000,38.530000,90.520000,80.620000
max,24.990148,100.000000,10.000000,2631.880000,28848.120000,28848.120000


# 1. Linear Regression

## (1) ~ ADI

- The coefficient: 0.611
    - If ADI increases (more disadvantaged), then time increases (less accessibility) by transit or walk.

In [12]:
adi_lm = smf.glm(formula = "access ~ ADI_NATRANK", data=bg_access)
adi_res = adi_lm.fit()
adi_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 access   No. Observations:               232337
Model:                            GLM   Df Residuals:                   232335
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                      1.1394e+05
Method:                          IRLS   Log-Likelihood:            -1.6823e+06
Date:                Thu, 11 May 2023   Deviance:                   2.6473e+10
Time:                        17:22:25   Pearson chi2:                 2.65e+10
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      48.0173      1.406     34.145      0.000      45.261      50.773
ADI_NATRANK     0.6106      0.024     25.047      0.000       0.563       0.658
===============================================================================
"""

## (2) ~ ADI + Rural

In [13]:
adi_rural_lm = smf.glm(formula = "access ~ ADI_NATRANK + C(status)", data=bg_access)
adi_rural_res = adi_rural_lm.fit()
adi_rural_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 access   No. Observations:               232336
Model:                            GLM   Df Residuals:                   232333
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                      1.1236e+05
Method:                          IRLS   Log-Likelihood:            -1.6806e+06
Date:                Thu, 11 May 2023   Deviance:                   2.6106e+10
Time:                        17:22:34   Pearson chi2:                 2.61e+10
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                53.4745      1.400     38.203      0.000      50.731      56.218
C(status)[T.Nonmetro]   112.7195      1.974     57.110      0.000     108.851     116.588
ADI_NATRANK               0.1237      0.026      4.821      0.000       0.073       0.174
=========================================================================================
"""